# Rank and Trade

### BUSI 722: Data-Driven Finance II
### Kerry Back, Rice University

### Outline

1. Build feature dataset with today's features
2. Load and apply model to predict
3. Trade to 140/40 portfolio with 100% in SPY
   (a) Close unwanted positions
   (b) Rebalance SPY
   (c) Open/rebalance long positions
   (d) Open/rebalance short positions
4. Save account equity and positions

In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from joblib import load 
import yfinance as yf 
from datetime import datetime
import os.path

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, GetAssetsRequest, AssetClass
from alpaca.trading.enums import OrderSide, TimeInForce

### Build Feature Dataset

- Don't need much history.  Start here in 2022.
- And don't need weekly returns (after computing momentum).

In [2]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'
driver = 'SQL+Server'
string = f"mssql+pyodbc://{username}:{password}@{server}/{database}" 
try: 
    conn = create_engine(string + "?driver='SQL+Server'").connect()
except:
    try:
        conn = create_engine(string + "?driver='ODBC+Driver+18+for+SQL+Server'").connect()
    except:
        import pymssql
        string = f"mssql+pymssql://{username}:{password}@{server}/{database}"   
        conn = create_engine(string).connect() 

In [3]:
sep_weekly = pd.read_sql(
    """ 
    select date, ticker, closeadj, closeunadj, volume, lastupdated from sep_weekly 
    where date >= '2022-01-01'
    order by ticker, date, lastupdated    
    """,
    conn,
)
sep_weekly = sep_weekly.groupby(["ticker", "date"]).last()
sep_weekly = sep_weekly.drop(columns=["lastupdated"])

ret = sep_weekly.groupby("ticker", group_keys=False).closeadj.pct_change()
ret.name = "ret"

price = sep_weekly.closeunadj
price.name = "price"

volume = sep_weekly.volume 
volume.name = "volume"

In [27]:
price.index.get_level_values("date").max()

datetime.date(2024, 3, 1)

In [4]:
ret_annual = sep_weekly.groupby("ticker", group_keys=False).closeadj.pct_change(52)
ret_monthly = sep_weekly.groupby("ticker", group_keys=False).closeadj.pct_change(4)
mom = (1 + ret_annual) / (1 + ret_monthly) - 1
mom.name = "mom"

In [5]:
weekly = pd.read_sql(
    """ 
    select date, ticker, pb, marketcap, lastupdated from weekly 
    where date>='2022-01-01'
    order by ticker, date, lastupdated    
    """,
    conn,
)
weekly = weekly.groupby(["ticker", "date"]).last()
weekly = weekly.drop(columns=["lastupdated"])

pb = weekly.pb
pb.name = "pb" 
marketcap = weekly.marketcap 
marketcap.name = "marketcap"

In [17]:
marketcap.tail()

ticker  date      
ZYXI    2024-02-02    396.0
        2024-02-09    421.1
        2024-02-16    430.9
        2024-02-23    437.7
        2024-03-01    417.0
Name: marketcap, dtype: float64

In [6]:
sf1 = pd.read_sql(
    """ 
    select datekey as date, ticker, assets, netinc, equity, lastupdated from sf1
    where datekey>='2022-01-01' and dimension='ARY' and assets>0 and equity>0
    order by ticker, datekey, lastupdated    
    """,
    conn,
)
sf1 = sf1.groupby(["ticker", "date"]).last()
sf1 = sf1.drop(columns=["lastupdated"])

# change dates to Fridays
from datetime import timedelta 
sf1 = sf1.reset_index()
sf1.date =sf1.date.map(
    lambda x: x + timedelta(4 - x.weekday())
)
sf1 = sf1.set_index(["ticker", "date"])
sf1 = sf1[~sf1.index.duplicated()]

assets = sf1.assets
assets.name = "assets" 
netinc = sf1.netinc 
netinc.name = "netinc" 
equity = sf1.equity
equity.name = "equity"

equity = equity.groupby("ticker", group_keys=False).shift() 
roe = netinc / equity 
roe.name = "roe"

assetgr = assets.groupby("ticker", group_keys=False).pct_change()
assetgr.name = "assetgr"

In [26]:
assetgr.index.get_level_values("date").max()

datetime.date(2024, 3, 8)

In [18]:
roe.tail()

ticker  date      
ZYME    2024-03-08   -0.240740
ZYNE    2022-03-04         NaN
        2023-03-31   -0.494065
ZYXI    2022-03-25         NaN
        2023-03-17    0.230612
Name: roe, dtype: float64

In [19]:
assetgr.tail()

ticker  date      
ZYME    2024-03-08   -0.104582
ZYNE    2022-03-04         NaN
        2023-03-31   -0.316005
ZYXI    2022-03-25         NaN
        2023-03-17   -0.126116
Name: assetgr, dtype: float64

In [22]:
df = pd.concat(
    (
        mom, 
        volume,
        price, 
        pb, 
        marketcap, 
        roe, 
        assetgr
        ), 
        axis=1
    )
df.tail()

,,mom,volume,price,pb,marketcap,roe,assetgr
ticker,date,,,,,,,
YORW,2024-03-08,NaN,NaN,NaN,NaN,NaN,0.114667,0.151999
YOTA,2022-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIMV,2022-01-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZM,2024-03-08,NaN,NaN,NaN,NaN,NaN,0.102707,0.221668
ZYME,2024-03-08,NaN,NaN,NaN,NaN,NaN,-0.240740,-0.104582


In [31]:
df = pd.concat(
    (
        mom, 
        volume,
        price, 
        pb, 
        marketcap, 
        roe, 
        assetgr
        ), 
        axis=1
    )
df["roe"] = df.groupby("ticker", group_keys=False).roe.ffill()
df["assetgr"] = df.groupby("ticker", group_keys=False).assetgr.ffill()

df = df.reset_index()
df.date = df.date.astype(str)
df = df[df.date=="2024-03-01"]
df = df[df.price >= 5]
df = df.dropna()

features = [
    "mom",
    "volume",
    "pb",
    "marketcap",
    "roe",
    "assetgr" 
]

In [32]:
df

,ticker,date,mom,volume,price,pb,marketcap,roe,assetgr
112,A,2024-03-01,-0.069450,1961886.0,139.06,6.9,40247.0,0.233742,0.021933
225,AA,2024-03-01,-0.462734,6671720.4,27.68,1.1,4885.8,-0.128251,-0.040729
1261,AAN,2024-03-01,-0.181575,592856.4,7.32,0.3,235.3,0.004060,-0.017158
1374,AAOI,2024-03-01,5.897436,4213505.4,15.41,2.9,613.6,-0.303504,-0.046727
1487,AAON,2024-03-01,0.176980,612802.4,83.26,9.3,6851.2,0.316780,0.156693
...,...,...,...,...,...,...,...,...,...
563792,ZUO,2024-03-01,-0.123752,2637974.0,8.01,9.1,1146.2,-1.160386,0.515229
564176,ZVRA,2024-03-01,0.072356,189195.4,7.08,3.8,256.4,-0.326807,-0.130975
564451,ZWS,2024-03-01,0.346221,655942.6,32.07,3.4,5515.9,0.069783,-0.068785
564606,ZYME,2024-03-01,0.373283,433029.6,12.51,2.1,875.7,0.499173,0.667108


In [30]:
df[df.date=="2024-03-01"]

,ticker,date,mom,volume,price,pb,marketcap,roe,assetgr


In [8]:
industries = pd.read_sql(
    """ 
    select ticker, famaindustry as industry from tickers   
    """,
    conn,
)
industries["industry"] = industries.industry.fillna("Almost Nothing")
df = df.merge(industries, on="ticker", how="left")
df = df.dropna()

In [9]:
for x in features:
    df[f"{x}_industry"] = df.groupby(
        ["industry"], 
        group_keys=False
    )[x].apply(
        lambda x: x - x.median()
    )

features += [f"{x}_industry" for x in features]

In [10]:
for f in features:
    df[f] = df[f].rank(pct=True)

### Load Model and Predict

In [11]:
model = load("mymodel.joblib")
df["predict"] = model.predict(df[features])

ValueError: Found array with 0 sample(s) (shape=(0, 12)) while a minimum of 1 is required by RandomForestRegressor.

### Best and worst stocks

- Best stocks must be tradable 
- Worst stocks must be tradable and shortable

In [ ]:
with open("keys.txt", "r") as f:
    keys = f.readlines()

key, secret_key = [x.strip() for x in keys]
trading_client = TradingClient(key, secret_key, paper=True)

search_params = GetAssetsRequest(asset_class=AssetClass.US_EQUITY)
assets = trading_client.get_all_assets(search_params)
tradable = [x.symbol for x in assets if x.tradable]
shortable = [x.symbol for x in assets if x.shortable]

In [ ]:
numstocks = 50

df = df.sort_values(by="predict", ascending=False)
best = df[["ticker", "predict"]].copy().reset_index(drop=True)
best = best[best.ticker.isin(tradable)].iloc[:numstocks]

df = df.sort_values(by="predict", ascending=True)
worst = df[["ticker", "predict"]].copy().reset_index(drop=True)
worst = worst[worst.ticker.isin(shortable)].iloc[:numstocks]

In [ ]:
best

,ticker,predict
0,SMCI,52.086654
1,COST,51.263532
2,ODFL,51.263532
3,CDNS,51.263532
4,VRSK,51.263532
5,CLX,51.263532
6,FAST,51.263532
7,SNPS,51.263532
8,ROL,51.263532
9,BAH,51.263532


In [ ]:
worst

,ticker,predict
2,EIGR,38.363996
6,PRPO,41.206560
12,BODY,43.961997
15,KPLT,44.359595
18,XOS,44.683329
19,CALC,44.738121
20,SKLZ,44.743086
22,ONCT,44.778882
27,AIRT,45.248374
29,TSE,45.379756


### Close unwanted positions

In [ ]:
positions = trading_client.get_all_positions()
positions = {x.symbol: float(x.qty) for x in positions}
positions_to_close = [
    symbol for symbol in positions 
    if (symbol not in best.ticker.to_list())
    and (symbol not in worst.ticker.to_list())
    and (symbol != "SPY")
]

for symbol in positions_to_close:
    qty = positions[symbol]
    order=MarketOrderRequest(
        symbol=symbol,
        qty=abs(qty),
        side=OrderSide.BUY if qty<0 else OrderSide.SELL,
        time_in_force=TimeInForce.DAY
    )
    _ = trading_client.submit_order(order)

### Rebalance SPY

In [ ]:
price = yf.download("SPY", start=2024, progress=False)["Close"].iloc[-1].item()

account = trading_client.get_account()
equity = float(account.equity)
qty = int(equity / price)
qty -= positions["SPY"] if "SPY" in positions else 0

if qty != 0:
    order = MarketOrderRequest(
        symbol="SPY",
        qty=abs(qty),
        side=OrderSide.BUY if qty>0 else OrderSide.SELL,
        time_in_force=TimeInForce.DAY
    )
    _ = trading_client.submit_order(order)

### Trade best stocks

In [ ]:
symbols = best.ticker.to_list()
prices = yf.download(symbols, start=2024)["Close"].iloc[-1]
symbols = [s for s in symbols if not np.isnan(prices[s])]
dollars = 0.4 * equity / numstocks
for symbol in symbols:
    price = prices[symbol]
    qty = int(dollars / price)
    qty -= positions[symbol] if symbol in positions else 0
    if qty != 0:
        try:
            order = MarketOrderRequest(
                symbol=symbol,
                qty=abs(qty),
                side=OrderSide.BUY if qty>0 else OrderSide.SELL,
                time_in_force=TimeInForce.DAY
            )
            _ = trading_client.submit_order(order)
        except Exception as error:
            print("An error occurred:", error)

[*********************100%%**********************]  50 of 50 completed
An error occurred: {"buying_power":"60.71","code":40310000,"cost_basis":"794.16","message":"insufficient buying power"}
An error occurred: {"buying_power":"60.71","code":40310000,"cost_basis":"762","message":"insufficient buying power"}
An error occurred: {"buying_power":"60.71","code":40310000,"cost_basis":"787","message":"insufficient buying power"}
An error occurred: {"buying_power":"60.71","code":40310000,"cost_basis":"721.44","message":"insufficient buying power"}
An error occurred: {"buying_power":"59.6","code":40310000,"cost_basis":"406.07","message":"insufficient buying power"}
An error occurred: {"buying_power":"59.33","code":40310000,"cost_basis":"785.7","message":"insufficient buying power"}
An error occurred: {"buying_power":"59.33","code":40310000,"cost_basis":"707.79","message":"insufficient buying power"}
An error occurred: {"buying_power":"59.31","code":40310000,"cost_basis":"783.55","message":"insuf

### Trade worst stocks

In [ ]:
symbols = worst.ticker.to_list()
prices = yf.download(symbols, start=2024)["Close"].iloc[-1]
symbols = [s for s in symbols if not np.isnan(prices[s])]
for symbol in symbols:
    price = prices[symbol]
    qty = - int(dollars / price)
    qty -= positions[symbol] if symbol in positions else 0
    if qty != 0:
        try:
            order = MarketOrderRequest(
                symbol=symbol,
                qty=abs(qty),
                side=OrderSide.BUY if qty>0 else OrderSide.SELL,
                time_in_force=TimeInForce.DAY
            )
            _ = trading_client.submit_order(order)
        except Exception as error:
            print("An error occurred:", error)

[*********************100%%**********************]  50 of 50 completed
An error occurred: {"buying_power":"405.43","code":40310000,"cost_basis":"819.18","message":"insufficient buying power"}


### Save data

In [ ]:
today = datetime.strftime(datetime.today(), "%Y-%m-%d")
account = trading_client.get_account()
equity = float(account.equity)
if os.path.isfile("equity.csv"):
    d = pd.read_csv("equity.csv", index_col="date")
    d.loc[today] = equity 
else: 
    d = pd.Series({today: equity})
    d.name = "equity" 
    d.index.name = "date"
d.to_csv("equity.csv")

In [ ]:
positions = trading_client.get_all_positions()
d = pd.DataFrame([x.qty for x in positions], index=[x.symbol for x in positions], columns=["shares"])
d["date"] = today
d.index.name = "symbol"
d = d.reset_index()
if os.path.isfile("positions.csv"):
    d0 = pd.read_csv("positions.csv")
    d = pd.concat((d0, d))
d.to_csv("positions.csv", index=False)

In [ ]:
df[df.date]